In [1]:
# Downloading necessary files
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import mplfinance as mpf 
import matplotlib.pyplot as plt
import datetime as dt
import datetime
from datetime import datetime
import plotly.graph_objects as go
import re
import json
import requests 
import codecs
import pandas as pd
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.offline as pyo


# display plotly in browser when run in Spyder
# pio.renderers.default = 'browser'

In [2]:
# stock =  input('Type in the stock symbol: ').upper()
stock = 'YPF'

In [4]:
start = "2009-01-01"
end = dt.datetime.now()
short_sma = 50
long_sma = 200

# creating a dataset for selected stock 
df = yf.download(stock, start, end,interval = '1d')
df = pd.DataFrame(df.dropna(), columns = ['Open', 'High','Low','Close', 'Adj Close', 'Volume'])


# preparing data from time series analysis
# eliminating any NAs 
df.index = pd.to_datetime(df.index)
df = df.asfreq('B')

print('============================================================')
print('Nan in each columns before Backfill for' , stock)
print('------------------------------------------------------------')  
print(df.isna().sum())
print('The df has {} rows and {} columns.'.format(*df.shape))

df = df.bfill()
print('============================================================')
print('Nan in each columns after Backfill for' , stock)
print('------------------------------------------------------------')  
print(df.isna().sum())
print('The df has {} rows and {} columns.'.format(*df.shape))
#    df = df.dropna()

# Create short SMA
df['short_SMA'] = df.iloc[:,1].rolling(window = short_sma).mean()

# Create Long SMA
df['long_SMA'] = df.iloc[:,1].rolling(window = long_sma).mean()

# Create daily_return
df['daily_return'] = df['Close'].pct_change(periods=1).mul(100)

# Create monthly_return
df['monthly_return'] = df['Close'].pct_change(periods=30).mul(100)

# Create annual_return
df['annual_return'] = df['Close'].pct_change(periods=365).mul(100)
df['Name'] = stock

print('============================================================')
print(stock, '- Full Dataset')
print('------------------------------------------------------------')
print(df.head())
print('------------------------------------------------------------')
print(stock, 'Full Dataset - Column Names')
print(df.columns)

print('============================================================')
print('Nan in each columns after Backfill for' , stock)
print('------------------------------------------------------------')  
print(df.isna().sum())
print('============================================================')

#     # write to csv
#     df.to_csv(r"df.csv", index =  True)


[*********************100%***********************]  1 of 1 completed
Nan in each columns before Backfill for YPF
------------------------------------------------------------
Open         118
High         118
Low          118
Close        118
Adj Close    118
Volume       118
dtype: int64
The df has 3398 rows and 6 columns.
Nan in each columns after Backfill for YPF
------------------------------------------------------------
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
The df has 3398 rows and 6 columns.
YPF - Full Dataset
------------------------------------------------------------
                 Open       High        Low      Close  Adj Close  Volume  \
Date                                                                        
2009-01-02  46.180000  46.180000  45.500000  45.900002  33.380840  3200.0   
2009-01-05  44.709999  45.000000  44.529999  45.000000  32.726315  4300.0   
2009-01-06  44.990002  47.270000  44.849998 

In [5]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume,short_SMA,long_SMA,daily_return,monthly_return,annual_return,Name
Date,,,,,,,,,,,,
2022-01-05,4.21,4.29,3.98,3.99,3.99,1949400.0,4.1028,4.55545,-4.086535,9.315066,-28.366250,YPF
2022-01-06,4.09,4.09,3.94,3.95,3.95,1342200.0,4.0946,4.55460,-1.002505,15.160349,-30.212011,YPF
2022-01-07,3.94,4.03,3.86,4.03,4.03,1133100.0,4.0876,4.55335,2.025321,17.492715,-29.545448,YPF
2022-01-10,4.02,4.03,3.87,3.91,3.91,1342500.0,4.0808,4.55210,-2.977670,15.339232,-28.124999,YPF
2022-01-11,3.95,4.11,3.90,4.09,4.09,2036200.0,4.0766,4.55070,4.603582,14.245817,-24.816174,YPF


In [6]:
# ZOOMING IN
# https://community.plotly.com/t/y-axis-autoscaling-with-x-range-sliders/10245/4
import plotly.graph_objs as go 


trace =  go.Candlestick(
                x = df.index,
                open = df["Open"],
                high = df["High"],
                low = df["Low"],
                close = df["Close"],
                name = stock)
short_sma = go.Scatter(x = df.index,
                             y = df['short_SMA'],
                             name = 'Short SMA 50-Day',
                             mode = 'lines',
                             customdata = df['Name'],
                             hovertemplate="<b>%{customdata}</b><br><br>" +
                                            "Date: %{x|%d %b %Y} <br>" +
                                            "Short (50-Day) Moving Average Price: %{y:$,.2f}<br>",
                             line = dict(color="blue"))

long_sma = go.Scatter(x = df.index,
                             y = df['long_SMA'],
                             name = 'Long SMA 200-Day',
                             mode = 'lines',
                             customdata = df['Name'],
                             hovertemplate="<b>%{customdata}</b><br><br>" +
                                            "Date: %{x|%d %b %Y} <br>" +
                                            "Long (200-Day) Moving Average Price: %{y:$,.2f}<br>",
                             line = dict(color="black"))

data = [trace, short_sma, long_sma]
layout = dict(
    title = 'Price of {}'.format(stock),
    yaxis_title = 'US Dollars',
    yaxis_tickprefix = '$', yaxis_tickformat = ',.2f',
    autosize = False, width = 800, height = 600,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                            dict(count = 7, step = "day", stepmode = "backward", label = "1W"),
                            dict(count = 1, step = "month", stepmode = "backward", label = "1M"),
                            dict(count = 3, step = "month", stepmode = "backward", label = "3M"),
                            dict(count = 6, step = "month", stepmode = "backward", label = "6M"),
                            dict(count = 1, step = "year", stepmode = "backward", label = "1Y"),
                            dict(count = 2, step = "year", stepmode = "backward", label = "2Y"),
                            dict(count = 5, step = "year", stepmode = "backward", label = "5Y"),
                            dict(count = 1, step = "all", stepmode = "backward", label = "MAX"),
                            dict(count = 1, step = "year", stepmode = "todate", label = "YTD")])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)

fig = go.FigureWidget(data=data, layout=layout)
# fig.update_layout(showlegend=False)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=-0.5,
    xanchor="right",
    x=1
))

fig.update_xaxes(
    rangeslider_visible=True,
    tickformatstops = [
        dict(dtickrange=[604800000, "M6"], value="%e. %b w"),
        dict(dtickrange=["M6", "M12"], value="%b '%y M"),
        dict(dtickrange=["M12", None], value="%Y Y")
    ],
    rangebreaks=[
        dict(bounds=["sat", "mon"]), #hide weekends
    ]
)


def zoom_price(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.Low.min() - 0.25, in_view.High.max() + 0.25]

fig.layout.on_change(zoom_price, 'xaxis.range')
fig

FigureWidget({
    'data': [{'close': array([45.90000153, 45.        , 47.        , ...,  4.03000021,  3.91000…